In [0]:
%sql
create schema if not exists bike_data.silver

In [0]:
df=spark.table('bike_data.bronze.prd_info_crm')

In [0]:
df.display()

### Checking Dupliactes

In [0]:
from pyspark.sql.functions import *

In [0]:
df.groupBy("prd_id") \
  .count() \
  .filter("count > 1") \
  .display()

In [0]:
df.describe()

In [0]:
df.count()

### Handling Null Values 

In [0]:
df.select(
    *[
        sum(col(c).isNull().cast("int")).alias(c)
        for c in df.columns
    ]
).display()

In [0]:
df.count()

In [0]:
df.describe()

#### Changing datatype

In [0]:
df=df.withColumn("prd_id",col("prd_id").cast("int"))


In [0]:
df.display()

In [0]:
df.schema

#### Changing Column names

In [0]:
for c in df.columns:
    if c.startswith("prd"):
        df = df.withColumnRenamed(c, c.replace("prd", "product", 1))


In [0]:
from pyspark.sql.functions import split, concat_ws

df =df.withColumn(
    "category_id",
    concat_ws("_",
        split(col("product_key"), "-")[0],
        split(col("product_key"), "-")[1]
    )
)


In [0]:
df.display()

In [0]:
# 10. Write Silver
df.write \
    .format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("bike_data.silver.product_details")